# How to handle connections to devices

## Automatically

By default, connections are handled automatically:

In [1]:
from nornir import InitNornir
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import napalm_get

In [2]:
nr = InitNornir(config_file="handling_connections/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=napalm_get,
    getters=["facts"]
)
print_result(r)

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Manually

In some circumstances, you may want to manage connections manually. To do so you can use 
[open_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.open_connection), [close_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connection), [close_connections](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connections) and [Nornir.close_connections](../ref/api/nornir.rst#nornir.core.Nornir.close_connections). For instance:

In [3]:
def task_manages_connection_manually(task):
    task.host.open_connection("napalm", configuration=task.nornir.config)
    r = task.run(
        task=napalm_get,
        getters=["facts"]
    )
    task.host.close_connection("napalm")
    
nr = InitNornir(config_file="handling_connections/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=task_manages_connection_manually,
)
print_result(r)

task_manages_connection_manually************************************************
* rtr00 ** changed : False *****************************************************
vvvv task_manages_connection_manually ** changed : False vvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_get ** changed : False --------------------------------------------- INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END task_manages_connection_manually ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Specifying connection parameters

When using the [open_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.open_connection) you can specify any parameters you want. If you don't, or if you let nornir open the connection automatically, nornir will read those parameters from the inventory. You can specify standard attributes at the object level if you want to reuse them across different connections or you can override them for each connection. For example:

In [4]:
!sed '2,35!d' ../../tests/inventory_data/hosts.yaml

dev1.group_1:
    port: 22
    hostname: dev1.group_1
    username:
    password: a_password
    platform: eos
    data:
        my_var: comes_from_dev1.group_1
        www_server: nginx
        role: www
        nested_data:
            a_dict:
                a: 1
                b: 2
            a_list: [1, 2]
            a_string: asdasd
    groups:
        - group_1
    connection_options:
        paramiko:
            port: 22
            hostname:
            username: root
            password: docker
            platform: linux
            extras: {}
        dummy:
            hostname: dummy_from_host
            port:
            username:
            password:
            platform:
            extras:
                blah: from_host
